In [1]:
# main.py
from initializations import Initializations
from trading_data import TradingData
from order_executor import OrderExecutor, get_token
from post_order_processor import PostOrderProcessor
import datetime
import time
import logging

# グローバルな設定
API_PASSWORD = "1995taka"
ORDER_PASSWORD = "1995tAkA@@"

#インデックス 295~300
# SYMBOL = "1579"
#NTT 156.8
SYMBOL = "9432"

EXCHANGE = 1  # 東証プライム市場を設定
SLEEP_INTERVAL = 0.3 # 待機時間（秒）

def setup_logger():
    """
    ロガーを設定します。
    """
    logger = logging.getLogger('TradingLogger')
    logger.setLevel(logging.INFO)
    
    # 既にハンドラーが設定されている場合は追加しない
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    # ロガーの伝播を無効化
    logger.propagate = False
    return logger

def main():
    # 初期化クラスのインスタンスを作成
    init = Initializations()
    init.order_password = ORDER_PASSWORD
    init.symbol = SYMBOL
    init.exchange = EXCHANGE
    # ロガーの設定
    init.logger = setup_logger()
    # トークンを取得
    token = get_token(API_PASSWORD)
    init.token = token
    # TradingDataクラスのインスタンスを作成
    trading_data = TradingData(init, token)
    # OrderExecutorクラスのインスタンスを作成
    order_executor = OrderExecutor(init, trading_data, token, ORDER_PASSWORD)
    # PostOrderProcessorクラスのインスタンスを作成
    post_order_processor = PostOrderProcessor(init)
    # 初回の価格取得
    initial_price = trading_data.fetch_current_price()
    init.previous_price = initial_price
    init.current_price = initial_price
    init.logger.info(f"初回価格を設定しました。価格: {initial_price}")
    # トレーディングループを開始
    while True:
        # 最新の株価データを取得
        current_price = trading_data.fetch_current_price()
        # price = (current_price)
        if current_price is not None:
            # データが4つになったらOHLCを生成
            if len(init.prices) >= 4:
                trading_data.create_ohlc()
                # Buy and Holdのエクイティカーブを計算
                trading_data.calculate_buy_and_hold_equity()
                # テクニカル指標の計算
                trading_data.calculate_technical_indicators()
                # 最新のテクニカル指標データを取得
                band_width = init.df['band_width']
                hist = init.df['hist']
                di_difference = init.df['di_difference']
                adx_difference = init.df['adx_difference']
                trading_data.update_latest_9_data(band_width, hist, di_difference, adx_difference)
                init.logger.info(f"Interpolated data length: {len(init.interpolated_data)}")
                # トレーディング状態の計算と更新
                post_order_processor.calculate_trading_values(datetime.datetime.now())
                # シグナルを生成
                trading_data.generate_signals(init.interpolated_data, init.R1, init.R2, init.R3, init.S1, init.S2, init.S3)
                # 注文の実行
                order_executor.execute_orders()
                # 補間データとOHLCデータを表示
                # trading_data.display_interpolated_data()
        # 指定した間隔（秒数）待機
        time.sleep(SLEEP_INTERVAL)

if __name__ == '__main__':
    main()

2025-01-21 15:13:30,461 - INFO - 取得した価格: 150.5
2025-01-21 15:13:30,462 - INFO - 初回価格を設定しました。価格: 150.5
2025-01-21 15:13:30,471 - INFO - 取得した価格: 150.5
2025-01-21 15:13:30,776 - INFO - 取得した価格: 150.5
2025-01-21 15:13:31,085 - INFO - 取得した価格: 150.5
c:\Users\user\dev\trading_system_v2.0\trading_data.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df1, df2_cleaned], ignore_index=True) # ignore_indexを変更
c:\Users\user\dev\trading_system_v2.0\trading_data.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before

[{'MarginTradeType': 3, 'ExecutionDay': 20250121, 'Expenses': 0.0, 'Commission': 0.0, 'CommissionTax': 0.0, 'ExpireDay': 20250121, 'ExecutionID': 'E2025012103E4I', 'AccountType': 4, 'Symbol': '9432', 'SymbolName': '日本電信電話', 'Exchange': 1, 'ExchangeName': '東証プ', 'Price': 150.5, 'LeavesQty': 100.0, 'HoldQty': 0.0, 'Side': '2', 'CurrentPrice': None, 'Valuation': None, 'ProfitLoss': None, 'ProfitLossRate': None}, {'MarginTradeType': 3, 'ExecutionDay': 20250121, 'Expenses': 0.0, 'Commission': 0.0, 'CommissionTax': 0.0, 'ExpireDay': 20250121, 'ExecutionID': 'E2025012103E4J', 'AccountType': 4, 'Symbol': '9432', 'SymbolName': '日本電信電話', 'Exchange': 1, 'ExchangeName': '東証プ', 'Price': 150.4, 'LeavesQty': 100.0, 'HoldQty': 0.0, 'Side': '1', 'CurrentPrice': None, 'Valuation': None, 'ProfitLoss': None, 'ProfitLossRate': None}]
買いの建玉ID: E2025012103E4I
売りの建玉ID: E2025012103E4J


INFO:order_executor:買いポジションに対する、逆指値返済注文の指値価格は 150.3 です。


買い価格 150.5
売り価格 150.4
買のほうが売よりも価格が高い
逆指値の買い決済価格 150.3
逆指値の売り決済価格 150.6


INFO:order_executor:売りポジションに対する、逆指値返済注文の指値価格は 150.6 です。
INFO:order_executor:買いポジション逆指値注文(20250121A01N01350808)が約定したため、売りポジション逆指値注文(20250121A01N01342603)をキャンセルします。
INFO:order_executor:ポジション取得に成功しました: 200 OK



====== 逆指値注文の監視開始 ======
  買い注文の逆指値返済注文ID: 20250121A01N01350808
  売り注文の逆指値返済注文ID: 20250121A01N01342603
🔔 注文状態の変化を検知
  買い注文: 約定
  売り注文: 約定

📈 買い注文 20250121A01N01350808 が約定
  → 売り注文 20250121A01N01342603 をキャンセル実行
HTTP Error 500: Internal Server Error
{'Code': 43, 'Message': '該当注文は既に約定済です'}
  キャンセル結果: None

====== 逆指値注文の監視終了 ======
総監視回数: 1

❌ アクティブなポジションが見つかりません


2025-01-21 15:30:44,196 - INFO - 取得した価格: 150.4
2025-01-21 15:30:44,500 - INFO - 取得した価格: 150.4
2025-01-21 15:30:44,808 - INFO - 取得した価格: 150.4
2025-01-21 15:30:45,115 - INFO - 取得した価格: 150.4
2025-01-21 15:30:45,165 - INFO - Interpolated data length: 17
INFO:order_executor:最初のサイクルでシグナルがありませんでした。
2025-01-21 15:30:45,478 - INFO - 取得した価格: 150.4
2025-01-21 15:30:45,784 - INFO - 取得した価格: 150.4
2025-01-21 15:30:46,093 - INFO - 取得した価格: 150.4
2025-01-21 15:30:46,402 - INFO - 取得した価格: 150.4
2025-01-21 15:30:46,460 - INFO - Interpolated data length: 18
INFO:order_executor:最初のサイクルでシグナルがありませんでした。
2025-01-21 15:30:46,771 - INFO - 取得した価格: 150.4
2025-01-21 15:30:47,079 - INFO - 取得した価格: 150.4
2025-01-21 15:30:47,384 - INFO - 取得した価格: 150.4
2025-01-21 15:30:47,693 - INFO - 取得した価格: 150.4
2025-01-21 15:30:47,753 - INFO - Interpolated data length: 19
INFO:order_executor:最初のサイクルでシグナルがありませんでした。
2025-01-21 15:30:48,082 - INFO - 取得した価格: 150.4
2025-01-21 15:30:48,388 - INFO - 取得した価格: 150.4
2025-01-21 15:30:48,699 - I

TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'